<a href="https://colab.research.google.com/github/JJang770/GitHub_tutorial/blob/main/Torchvision_tr02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
!pip3 install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

from PIL import Image


In [ ]:
print(f'torch version:{torch.__version__}')
print(f'torchvision version:{torchvision.__version__}')

In [ ]:
model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True).eval()

In [ ]:
summary(model)

In [ ]:
TF = transforms.Compose([transforms.ToTensor()])

In [ ]:
IMG_SIZE=480
THRESHOLD=0.95

In [ ]:
imgRaw = Image.open('/content/drive/MyDrive/fastcampus_vision/imgs/07.jpg')
print(f'imgRaw type:{type(imgRaw)}')
print(f'imgRaw width:{imgRaw.width} height:{imgRaw.height}]\n')
img = imgRaw.resize((IMG_SIZE,int(imgRaw.height*IMG_SIZE/imgRaw.width)))
print(f'img type:{type(img)}')
print(f'img width:{img.width} height:{img.height} size:{img.size}')


In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(img)

In [ ]:
imgxx = TF(img)
print(f'imgxx type:{type(imgxx)}, shape:{imgxx.shape}\n')
imgX = imgxx.unsqueeze(0)
print(f'imgX type:{type(imgX)}, shape:{imgX.shape}')

In [ ]:
out = model(imgX)[0]
print(type(out), out.keys())

In [ ]:
codes =[
    Path.MOVETO,
    Path.LINETO,
    Path.LINETO
]

fig, ax = plt.subplots(1,figsize=(16,16))
ax.imshow(img)

for box, score, keypoints in zip(out['boxes'], out['scores'], out['keypoints']):
  score = score.detach().numpy()
  if (score < THRESHOLD):
    continue

  box = box.detach().numpy()
  print(f'keypoints original shape:{keypoints.shape}')
  keypoints = keypoints.detach().numpy()[:,:2]
  print(f'keypoints final shape:{keypoints.shape}')

  #draw boxes
  pxRect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=1,edgecolor='b',facecolor='none')
  ax.add_patch(pxRect)

  #draw keypoints
  for p in keypoints:
    pxCircle = patches.Circle((p[0],p[1]),radius=2,facecolor='r')
    ax.add_patch(pxCircle)

  #draw left arm
  path = Path(keypoints[5:10:2],codes)
  pxPath = patches.PathPatch(path,linewidth=2,edgecolor='r',facecolor='none')
  ax.add_patch(pxPath)

  #draw right arm
  path = Path(keypoints[6:11:2],codes)
  pxPath = patches.PathPatch(path,linewidth=2,edgecolor='r',facecolor='none')
  ax.add_patch(pxPath)

  #draw left leg
  path = Path(keypoints[11:16:2],codes)
  pxPath = patches.PathPatch(path,linewidth=2,edgecolor='r',facecolor='none')
  ax.add_patch(pxPath)

  #draw right leg
  path = Path(keypoints[12:17:2],codes)
  pxPath = patches.PathPatch(path,linewidth=2,edgecolor='r',facecolor='none')
  ax.add_patch(pxPath)